In [63]:
import spectral.io.envi as envi
from proj.tasks import send_matrix_to_queue_task, read_image_spectral, clf_RF_1, read_col_data_from_file, load_sklearn_model_to_file, get_full_path
import numpy as np
import pandas as pd
import sqlite3
from concurrent.futures import ThreadPoolExecutor, as_completed
import os
import json
from datetime import datetime
import time
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.colors import LinearSegmentedColormap, ListedColormap
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from matplotlib import colormaps
import time 
import dill
from torch import nn
import torch

In [64]:
# Load object file to code
def load_sklearn_model_to_file_by_cloudpickle(file_path):
    with open(file_path, 'rb') as f:
        return dill.load(f)

In [65]:
# Read spectral image
img = read_image_spectral()

In [66]:
# Get 
NUM_ROW, NUM_COL, NUM_BAND = img.shape
BANDS_ZERO = np.zeros((122), dtype=np.uint8)

In [67]:
NUM_ROW

6478

In [68]:
NUM_COL

5287

In [69]:
def col_data(col):
    file_path = f"./data/img_col_data/img_{col}.npz"
    return read_col_data_from_file(file_path)

In [70]:
def get_position_of_bands_zero(col_data):
    matching_rows = np.where(np.all(col_data == BANDS_ZERO, axis=1))
    return matching_rows[0]

In [71]:

def predict_target_value_for_img(model, saved_result_path):
    if os.path.exists(saved_result_path):
        print(f"File {saved_result_path} already exists.")
        with np.load(saved_result_path) as file:
            return file["arr_0"]
    
    print(f"File {saved_result_path} NOT exists.")
    target_value = np.zeros((NUM_ROW, NUM_COL))
    for col in range(NUM_COL):
        if col % 100 == 0:
            print(col)
        file_path = f"./data/img_col_data/img_{col}.npz"
        bands = read_col_data_from_file(file_path)
        bands_ix = model.bands_ix

        if issubclass(type(model), nn.Module):
            X_target = bands[:, bands_ix].astype(np.float32)
            target_value[:, col] = model.predict(torch.tensor(X_target)).detach().numpy().reshape(X_target.shape[0])
        else:
            target_value[:, col] = model.predict(bands[:, bands_ix])

        band_zeros_position = get_position_of_bands_zero(bands)
        target_value[band_zeros_position, col] = 0
    np.savez_compressed(saved_result_path, target_value)
    return target_value

In [72]:
def get_target_matrix(model_file_path, output_file_path):
    model = load_sklearn_model_to_file_by_cloudpickle(model_file_path)
    if issubclass(type(model), nn.Module):
        model.predict = lambda x: model(x)
    target_values = predict_target_value_for_img(model, output_file_path)
    return target_values
    

In [ ]:
list_model_path = [
    # r"D:\code\model_saved\RF_save\J_predict_P_2024-03-05-23-26-37_using_get_average_bands.pkl",
    # r"D:\code\model_saved\RF_save\T_predict_K_2024-03-02-13-32-08_using_get_average_bands.pkl",
    # r"D:\code\model_saved\DT_save\J_predict_P_2024-03-05-18-46-25_using_get_average_bands.pkl",
    # r"D:\code\model_saved\DT_save\T_predict_K_2024-03-02-13-03-33_using_get_average_bands.pkl",
    r"D:\code\model_saved\NN_save\NN_object\J_predict_P_2024-03-04-22-03-03_using_get_max_bands.pkl",
    # r"D:\code\model_saved\NN_save\NN_object\J_predict_K_2024-03-05-10-33-28_using_get_max_bands.pkl",
]

for model_file_path in list_model_path:
    file_name = os.path.splitext(os.path.basename(model_file_path))[0]
    output_file_path = f"./data/saved_result/{file_name}_1.npz"
    target_value = get_target_matrix(model_file_path, output_file_path)
    print(target_value.shape)

In [ ]:
# tensor([[5885.6289],
#         [7505.2896],
#         [6279.8838],
#         [6624.3481]])

In [77]:
target_value

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
# row	col
# BC2	3778	1808
# BC3	4641	3068
# BC4	3185	4085
# BC5	2468	4010

In [80]:
BC2 = target_value[3778, 1808]
BC3 = target_value[4641, 3068]
BC4 = target_value[3185, 4085]
BC5 = target_value[2468, 4010]

In [81]:
BC2, BC3, BC4, BC5

(15594.82421875, 5862.5341796875, 5531.50830078125, 2353.9033203125)

In [74]:
model = load_sklearn_model_to_file_by_cloudpickle(r"D:\code\model_saved\NN_save\NN_object\J_predict_P_2024-03-04-22-03-03_using_get_max_bands.pkl")

In [75]:
model

NeutralNetwork(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=48, out_features=100, bias=True)
    (1): ReLU()
    (2): Linear(in_features=100, out_features=100, bias=True)
    (3): ReLU()
    (4): Linear(in_features=100, out_features=100, bias=True)
    (5): ReLU()
    (6): Linear(in_features=100, out_features=100, bias=True)
    (7): ReLU()
    (8): Linear(in_features=100, out_features=100, bias=True)
    (9): ReLU()
    (10): Linear(in_features=100, out_features=100, bias=True)
    (11): ReLU()
    (12): Linear(in_features=100, out_features=100, bias=True)
    (13): ReLU()
    (14): Linear(in_features=100, out_features=100, bias=True)
    (15): ReLU()
    (16): Linear(in_features=100, out_features=1, bias=True)
  )
)